In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

In [ ]:
# 指定要分析的已经导出的聊天记录
input_file = ...

data = pd.read_csv('final_data.tsv', sep='\t')
data.head()

In [ ]:
# 消息量统计
print('我们一共发了{}条信息'.format(len(data)))
print('我发了{}条信息'.format(len(data[data['isSend'] == 1])))

# 指定你要分析的人的名字
talker = ...

print('{}发了{}条信息'.format(talker, len(data[data['isSend'] == 0])))

In [ ]:
# 平均消息量
length = np.mean(data[data['isSend'] == 1].content.str.len())
print('我发的信息平均字数为{}'.format(round(length, 2)))
length = np.mean(data[data['isSend'] == 0].content.str.len())
print('{}发的信息平均字数为{}'.format(talker, round(length, 2)))

In [ ]:
# 调整time的格式
data['time'] = pd.to_datetime(data.time, format='%Y-%m-%d %H:%M:%S', errors='ignore')

# 根据time设置每条消息是星期几
data['weekday'] = data.time.dt.weekday
data.weekday.describe()

# 根据time设置每条消息是几几年
data['year'] = data.time.dt.year
data.year.describe()

# 根据time设置每条消息是什么日期
data['date'] = data.time.dt.date

# 根据time设置每条消息是什么小时
data['hour'] = data.time.dt.hour

In [ ]:
print('本次数据截至'.format(data.date.sort_values()[:1]))
data.date.sort_values(ascending=False)[:1]

In [ ]:
print('我们平均每天发{}条消息'.format(round(len(data)/len(data.date.unique()), 0)))

In [ ]:
# create heatmap of the counts over the years
# 制作聊天记录的热点图，颜色越深，聊天消息越多
for year in data.year.unique():
    piv = pd.pivot_table(data[data['year'] == year], values="content",index=["hour"], columns=["date"], aggfunc=lambda x: len(x), fill_value=0)
    fig, ax = plt.subplots(figsize=(80,10))
    ax.set_title('我们{}年的消息量按小时分布'.format(year), fontproperties=font)
    ax = sns.heatmap(piv, cmap="YlGnBu")
    plt.show()
    fig.savefig('{}.png'.format(year))

In [ ]:
# 统计一个星期里星期几聊天最多
weekday_count = data.weekday.value_counts().sort_index()
fig, ax = plt.subplots(nrows=1, ncols=1)
font = fm.FontProperties(fname='FZQKBYSJW.TTF')
ax.bar(['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期天'], weekday_count)
ax.set_xticklabels(['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期天'], fontproperties=font)
ax.set_title('我们最经常在星期几发信息？', fontproperties=font)
fig.savefig('weekday.png')

In [ ]:
# 统计一个星期里星期几我发的消息最多
my_weekday_count = data[data.isSend == 1].weekday.value_counts().sort_index()
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_title('我星期几发的消息最多？', fontproperties=font)
ax.bar(['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期天'], my_weekday_count)
ax.set_xticklabels(['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期天'], fontproperties=font)
fig.savefig('weekday_me.png')

In [ ]:
# 统计一个星期里星期几对方发的消息最多
# 设置对方名字，保存图的时候用
talker = ...

weekday_count = data[data.isSend == 0].weekday.value_counts().sort_index()
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_title('他星期几发的消息最多？', fontproperties=font)
ax.bar(['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期天'], weekday_count)
ax.set_xticklabels(['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期天'], fontproperties=font)
fig.savefig('weekday_{}.png'.format(talker))

In [ ]:
# 统计几几年我们发的消息字数最多
data['length'] = data.content.str.len()
year_len_sum = data[data.type == 1].groupby('year').sum().length
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_title('我们几几年发的的字数？', fontproperties=font)
ax.bar(year_len_sum.index, year_len_sum)
plt.xticks(year_len_sum.index)
fig.savefig('years_len.png')

In [ ]:
# 统计几几年我们发的表情最多
year_count = data[data.type == 47].year.value_counts().sort_index()
print(year_count)
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_title('我们几几年发的表情最多？', fontproperties=font)
ax.bar(year_count.index, year_count)
plt.xticks(year_count.index)
fig.savefig('plots/years_emoji.png'.format(year))

In [ ]:
# 统计几几年我们说话说得最多的日期TOP 10
year = ...

print('我们在{}年说话说得最多的日期TOP 10'.format(year))
print(data[data.year==year].date.value_counts().sort_values(ascending=False)[:10])

In [ ]:
# 统计几几年我们说话说得最少的日期TOP 10
year = ...

print('我们在{}年说话说得最少的日期TOP 10'.format(year))
print(data[data.year==year].date.value_counts().sort_values(ascending=True)[:10])

In [ ]:
# 分析每一年每一天的消息量
for year in data.year.sort_values().unique():
    date_count = data[data.year == year].date.value_counts()
    fig, ax = plt.subplots(figsize=(20,5))
    ax.set_title('我们{}年的消息量分布'.format(year), fontproperties=font)
    ax.bar(date_count.index, date_count)
    fig.savefig('date_{}.png'.format(year))

In [ ]:
# 分析这么多年来，24小时每小时的总消息量
hour_count = data.hour.value_counts().sort_index()
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_title('我们什么时间发的消息最多？', fontproperties=font)
ax.bar(hour_count.index, hour_count)
plt.xticks(hour_count.index)
fig.savefig('hours.png'.format(year))

In [ ]:
# 统计一些词我们说过的次数

# 设置对方的名字，输出时用
talker = ...
# 设置词的list
word_list = [...]

def count_word(word):
    print('我们一共说了{}句{}'.format(len(data[data.content.str.contains(word)]), word))
    print('我说了{}句{}'.format(len(data[(data.isSend == 1) & (data.content.str.contains(word))]), word))
    print('{}说了{}句{}'.format(len(data[(data.isSend == 0) & (talker, data.content.str.contains(word))]), word))

    
for word in word_list:
    print()
    count_word(word)